# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [ ]:
%store -r

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [ ]:
# Import libraries
import geopandas as gpd # work with vector data
import pandas as pd # work with tabular data

import hvplot.pandas # to make interactive plots
import rioxarray as rxr # to work with raster data
import xarray as xr # to work with data arrays

# STEP 4: Is the NDVI different within the **Gila River Indian Community** after the **return of water rights**?

You will compute the mean NDVI inside and outside the fire boundary.
First, use the code below to get a `GeoDataFrame` of the area outside
the Reservation.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Check the variable names - Make sure that the code uses your
boundary <code>GeoDataFrame</code></li>
<li>How could you test if the geometry was modified correctly? Add some
code to take a look at the results.</li>
</ol></div></div>

In [ ]:
# Compute the area outside the Gila River Indian Community boundary

outside_gila_gdf = (
    gpd.GeoDataFrame(geometry=boundary_gdf.envelope)
    .overlay(boundary_gdf, how='difference'))

outside_gila_gdf.plot()

Next, clip your DataArray to the boundaries for both inside and outside
the reservation. You will need to replace the `GeoDataFrame` name with
your own. Check out the [lesson on clipping data with the `rioxarray`
library in the
textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/).

> **GOTCHA ALERT**
>
> It’s important to use `from_disk=True` when clipping large arrays like
> this. It allows the computer to use less valuable memory resources
> when clipping - you will probably find that otherwise the cell below
> crashes your kernel.

In [ ]:
# Clip NDVI data to both inside and outside the boundary
ndvi_inside = ndvi_da.rio.clip(boundary_gdf.geometry, from_disk=True)
print(ndvi_inside)

ndvi_outside = ndvi_da.rio.clip(outside_gila_gdf.geometry, from_disk=True)
print(ndvi_outside)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>For <strong>both inside and outside</strong> the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary:</p>
<ul>
<li>Group the data by year</li>
<li>Take the mean. You always need to tell reducing methods in
<code>xarray</code> what dimensions you want to reduce. When you want to
summarize data across <strong>all</strong> dimensions, you can use the
<code>...</code> syntax, e.g. <code>.mean(...)</code> as a
shorthand.</li>
<li>Select the NDVI variable</li>
<li>Convert to a DataFrame using the <code>to_dataframe()</code>
method</li>
<li>Join the two DataFrames for plotting using the <code>.join()</code>
method. You will need to rename the columns using the
<code>lsuffix=</code> and <code>rsuffix=</code> parameters</li>
</ul>
<p>Finally, plot annual July means for both inside and outside the
Reservation on the same plot.</p></div></div>

> **GOTCHA ALERT**
>
> The DateIndex in pandas is a little different from the Datetime
> Dimension in xarray. You will need to use the `.dt.year` syntax to
> access information about the year, not just `.year`.

In [ ]:
# Compute mean annual July NDVI
july_ndvi_inside_df = (ndvi_inside
                       .groupby(ndvi_inside.date.dt.year)
                       .mean(...)
                       .NDVI.to_dataframe())
print('Mean NDVI inside GRIC\n', july_ndvi_inside_df.head())

july_ndvi_outside_df = (ndvi_outside
                       .groupby(ndvi_outside.date.dt.year)
                       .mean(...)
                       .NDVI.to_dataframe())
print('Mean NDVI outside GRIC\n', july_ndvi_outside_df.head())

In [ ]:
# Join inside and outside GRIC df and rename columns
july_ndvi_df = (july_ndvi_inside_df[['NDVI']]
                .join(july_ndvi_outside_df[['NDVI']], 
                      lsuffix=' Inside GRIC', 
                      rsuffix=' Outside GRIC'))
july_ndvi_df

In [ ]:
# Plot mean NDVI inside and outside GRIC boundary
july_ndvi_df.hvplot(title='Mean July NDVI Inside and Outside GRIC')

Now, take the difference between outside and inside the site boundary
and plot that. What do you observe? Don’t forget to write a headline and
description of your plot!

In [ ]:
# Plot difference inside and outside the boundary
july_ndvi_df['Difference'] = (july_ndvi_df['NDVI Outside GRIC'] 
                              - july_ndvi_df['NDVI Inside GRIC'])
july_ndvi_df

In [ ]:
# Plot difference inside and outside the boundary
july_ndvi_df.Difference.hvplot(title='Difference in NDVI within and outside GRIC')

The above plot visualizes the difference in vegetation health change from 2001 - 2022 between the outside of the Gila River Indian Community and the inside of the Gila River Indian Community. It demonstrates that this difference in NDVI between the outside and inside of the community boundary has decreased over time since the restoration of water rights to the community in 2004. This means that while the outside and inside of the boundary contain different NDVI values, either the vegetation within the Gila River Indian Community has become more healthy on average to decrease the gap between it and the outside NDVI values, OR the vegetation outside the Gila River Indian Community has become less healthy on average to decrease this gap. While the first option corresponds more with what is displayed in the previous plot titled "Mean July NDVI Inside and Outside GRIC," a combination of both has most likely produced this effect seen in the difference plot.

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [ ]:
%store var1 var2

Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!